In [1]:
# %pip install opencv-python deepface mysql-connector-python numpy scipy

In [2]:
import cv2
from deepface import DeepFace
import mysql.connector
import os
import time
from datetime import datetime
import socket
import numpy as np

# ==== Configuration ====


In [3]:
CHECK_INTERVAL = 20                # Time between each camera activation (seconds)
MAX_CAMERA_USE_TIME = 5           # Duration to use camera during each activation (seconds)
CAPTURED_FACES_DIR = "captured_faces"  # Directory to save captured face snapshots

# ==== Ensure storage directory exists ====


In [4]:
def ensure_dir(directory):
    """Creates directory if it doesn't exist."""
    if not os.path.exists(directory):
        os.makedirs(directory)

# ==== Main Background Face Recognition Class ====


In [5]:
class BackgroundFaceCapture:
    def __init__(self):
        self.db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="emotion_detection"
        )
        self.cursor = self.db.cursor()
        self.pc_name = socket.gethostname()
        ensure_dir(CAPTURED_FACES_DIR)

    def save_snapshot(self, face_img, face_id, timestamp):
        """Saves a snapshot of the face into the captured_faces/ directory with timestamp."""
        font = cv2.FONT_HERSHEY_SIMPLEX
        timestamp_text = f"Captured: {timestamp}"
        cv2.putText(face_img, timestamp_text, (10, 30), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

        filename = f"{face_id}_{timestamp}.jpg"
        path = os.path.join(CAPTURED_FACES_DIR, filename)
        saved = cv2.imwrite(path, face_img)
        if saved:
            print(f"📸 Snapshot saved: {path}")
        else:
            print("❌ Failed to save snapshot")

    def process_frame(self, frame):
        """Detects and captures face snapshots with full head framing and glasses handling."""
        try:
            faces = DeepFace.extract_faces(frame, enforce_detection=False)
            if not faces:
                print("⚠️ No face detected.")
                return None

            for face in faces:
                area = face['facial_area']
                x = area['x']
                y = area['y']
                w = area['w']
                h = area['h']

                # Add padding to include full head and avoid cropping glasses
                pad_y = int(h * 0.6)
                pad_x = int(w * 0.4)
                x1 = max(0, x - pad_x)
                y1 = max(0, y - pad_y)
                x2 = min(frame.shape[1], x + w + pad_x)
                y2 = min(frame.shape[0], y + h + pad_y)

                face_img = frame[y1:y2, x1:x2]

                # Enhance clarity under glasses using CLAHE
                gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                enhanced_gray = clahe.apply(gray)
                face_img_clahe = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)

                # Save the face snapshot
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                face_id = f"{self.pc_name}_user"
                self.save_snapshot(face_img_clahe, face_id, timestamp)

        except Exception as e:
            print(f"⚠️ Face processing error: {e}")

    def _try_get_camera(self):
        """Attempts to access an available camera."""
        for i in range(3):
            cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)
            if cap.isOpened():
                return cap
        print("❌ No available camera found.")
        return None

    def run(self):
        """Main loop to run the background face capture."""
        print("🔄 Background face capture started...")
        while True:
            cap = self._try_get_camera()
            if cap:
                start_time = time.time()
                while time.time() - start_time < MAX_CAMERA_USE_TIME:
                    ret, frame = cap.read()
                    if ret:
                        self.process_frame(frame)
                cap.release()
            time.sleep(CHECK_INTERVAL)

# ==== Start the service ====


In [ ]:
if __name__ == "__main__":
    recognizer = None
    try:
        recognizer = BackgroundFaceCapture()
        recognizer.run()
    except KeyboardInterrupt:
        print("🛑 Background service stopped manually.")
    finally:
        if recognizer is not None:
            recognizer.db.close()


🔄 Background face capture started...
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203228.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203228.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203229.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203230.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203231.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203253.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203254.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203255.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203256.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203257.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203258.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203321.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203321.jpg
📸 Snapshot saved: captured_faces\Piss-Off_user_20250523_203322.jpg
📸 Snapshot saved: capture